In [1]:
import torch

In [2]:
!pip install git+https://github.com/ChaoningZhang/MobileSAM.git

  Cloning https://github.com/ChaoningZhang/MobileSAM.git to /private/var/folders/cg/l243p36n10s5v1x_vrt2wzs80000gn/T/pip-req-build-dndm0vfc
  Running command git clone -q https://github.com/ChaoningZhang/MobileSAM.git /private/var/folders/cg/l243p36n10s5v1x_vrt2wzs80000gn/T/pip-req-build-dndm0vfc
You should consider upgrading via the '/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/venv/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

model_type = "vit_t"
sam_checkpoint = "./weights/mobile_sam.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"

mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()

/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/venv/lib/python3.8/site-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/venv/lib/python3.8/site-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/venv/lib/python3.8/site-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_21m_224 in registry with mo

FileNotFoundError: [Errno 2] No such file or directory: './weights/mobile_sam.pt'

In [ ]:
from mobile_sam import SamAutomaticMaskGenerator

In [ ]:
import cv2
import supervision as sv

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
IMAGE_NAME = "dog.jpeg"
IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)

In [ ]:
image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(mobile_sam)
# masks = mask_generator.generate(image_bgr)

In [ ]:
# helper function that loads an image before adding it to the widget

import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [ ]:
# from jupyter_bbox_widget import BBoxWidget
#
# widget = BBoxWidget()
# widget.image = encode_image(IMAGE_PATH)
# widget

In [ ]:
import numpy as np

default_box = {'x': 68, 'y': 247, 'width': 555, 'height': 678, 'label': ''}

# box = widget.bboxes[0] if widget.bboxes else default_box
box = default_box
box = np.array([
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
])

In [ ]:
predictor = SamPredictor(mobile_sam)
predictor.set_image(image_bgr)
masks, _, _ = predictor.predict(
    box=box,
    multimask_output=False
)

In [ ]:
box_annotator = sv.BoxAnnotator(color=sv.Color.red())
mask_annotator = sv.MaskAnnotator(color=sv.Color.red(), color_lookup=sv.ColorLookup.INDEX)

detections = sv.Detections(
    xyxy=sv.mask_to_xyxy(masks=masks),
    mask=masks
)
detections = detections[detections.area == np.max(detections.area)]

source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections, skip_label=True)
segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[source_image, segmented_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)